In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy import units as u
import pandas as pd
from collections import OrderedDict

In [2]:
from lsst.geom import Point2D, Point2I
from lsst.afw.cameraGeom import FIELD_ANGLE, PIXELS
import lsst.afw.cameraGeom.utils as cgUtils
from lsst.daf.persistence import Butler, NoResults

In [3]:
import lsst.syseng.throughputs as st
from lsst.sims.photUtils import PhotometricParameters, Bandpass
from lsst.sims.utils import angularSeparation

In [4]:
rname = 'R10'
#rname = 'R22'
#rname = 'R01'

### Default photometric parameters, as used in standard m5 calculations
Note that effarea is not in this list here, because it varies with field.

The read noise is not in this list either, because it varies by amp.

In [5]:
exptime=15 
nexp=2
othernoise=0 
darkcurrent=0.2
X=1.0

### Set up throughputs for hardware and atmosphere. Use the default detector QE as in syseng_throughput for now

In [6]:
# Add losses to each component?
addLosses = True
defaultDirs = st.setDefaultDirs()
defaultDirs

{'detector': '/home/bxin/notebooks/syseng_throughputs/components/camera/detector/joint_minimum',
 'lens1': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens1',
 'lens2': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens2',
 'lens3': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens3',
 'filters': '/home/bxin/notebooks/syseng_throughputs/components/camera/filters',
 'mirror1': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror1',
 'mirror2': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror2',
 'mirror3': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror3',
 'atmosphere': '/home/bxin/notebooks/syseng_throughputs/siteProperties'}

In [7]:
atmos = st.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
mirror1 = st.buildMirror(defaultDirs['mirror1'], addLosses)
mirror2 = st.buildMirror(defaultDirs['mirror2'], addLosses)
mirror3 = st.buildMirror(defaultDirs['mirror3'], addLosses)
lens1 = st.buildLens(defaultDirs['lens1'], addLosses)
lens2 = st.buildLens(defaultDirs['lens2'], addLosses)
lens3 = st.buildLens(defaultDirs['lens3'], addLosses)
filters = st.buildFilters(defaultDirs['filters'], addLosses)

detector0 = st.buildDetector(defaultDirs['detector'], addLosses) #design QE
detector = Bandpass()

### Butler access to the QE data

In [8]:
DATADIR = f"{os.environ['OBS_LSST_DIR']}/lsstcam/CALIB" 
print(DATADIR)
butler = Butler(DATADIR)
cam = butler.get('camera')

/home/bxin/lsst_stack/obs_lsst/lsstcam/CALIB


### Lay out the grid that covers the entire focal plane

In [ ]:
nside = 3
raDeg, decDeg = np.meshgrid(np.linspace(-2.1,2.1, nside), np.linspace(-2.1,2.1, nside))
ra=np.radians(raDeg)
dec = np.radians(decDeg)

In [117]:
# mapping based on 
#https://confluence.slac.stanford.edu/pages/viewpage.action?spaceKey=LSSTCAM&title=Raft+Delivery+and+Acceptance+Testing+Status
dd = pd.read_csv('raftInstall.csv',index_col=0)

### Prepare vignetting function

In [27]:
# below we use v3.11 values
vfile = f"{os.environ['HOME']}/notebooks/f_factors/data/vignettingF.txt"
M1D = 8.36 #clear aperture as in Optical design
aa = np.loadtxt(vfile, skiprows=12)
vr = aa[:,0]
vv = aa[:,1]

In [ ]:
#This will not be needed after DM-22605 gets merged
from lsst.obs.lsst.lsstCamMapper import LsstCamMapper
mapper = LsstCamMapper()
lsstcam = mapper.camera

In [157]:
filterlist = tuple([s for s in filters])
alist = ('raDeg', 'decDeg', 'radDeg', 'effarea', 'readnoise')
detectors = []
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
    detectors.append(det.getName())
adf = pd.DataFrame(index=alist, columns=detectors, dtype=object)
m5df = pd.DataFrame(index=filterlist, columns=detectors, dtype=object)

In [162]:
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
    raDeg = {}
    decDeg = {}
    readnoise = {}
    for amp in det:
        i = amp.getName()
        amp_point = amp.getBBox().getCenter()
        raDec = det.transform(amp_point, PIXELS, FIELD_ANGLE) 
        [raDeg[i], decDeg[i]] = np.degrees(raDec)
        readnoise[i] = amp.getReadNoise()
    key = rname+'_'+dname
    adf[key].loc['raDeg'] = list(OrderedDict(sorted(raDeg.items())).values())
    adf[key].loc['decDeg'] = list(OrderedDict(sorted(decDeg.items())).values())
    adf[key].loc['readnoise'] = list(OrderedDict(sorted(readnoise.items())).values())
        
    #effetive area
    radius = angularSeparation(0., 0., adf[key]['raDeg'], adf[key]['decDeg'])
    adf[key].loc['radDeg'] = radius
    adf[key].loc['effarea'] = np.interp(radius, vr, vv)*np.pi*(M1D/2)**2

In [163]:
adf

,R10_S00,R10_S01,R10_S02,R10_S10,R10_S11,R10_S12,R10_S20,R10_S21,R10_S22
raDeg,"[-1.74775, -1.719472222222222, -1.691194444444...","[-1.5128611111111108, -1.4845833333333336, -1....","[-1.278027777777778, -1.2497500000000001, -1.2...","[-1.7474722222222223, -1.7191944444444447, -1....","[-1.5123611111111108, -1.4840833333333332, -1....","[-1.2774722222222221, -1.2491944444444445, -1....","[-1.7467499999999998, -1.7184722222222222, -1....","[-1.5118055555555554, -1.4835277777777778, -1....","[-1.2770277777777779, -1.24875, -1.22047222222..."
decDeg,"[-0.8822222222222221, -0.8822222222222221, -0....","[-0.8829444444444443, -0.8829444444444442, -0....","[-0.8832222222222221, -0.883222222222222, -0.8...","[-0.6473888888888889, -0.6473888888888889, -0....","[-0.6478333333333333, -0.6478333333333333, -0....","[-0.6483888888888889, -0.648388888888889, -0.6...","[-0.4125, -0.4124999999999999, -0.412499999999...","[-0.4129444444444443, -0.41294444444444434, -0...","[-0.41338888888888886, -0.4133888888888888, -0..."
radDeg,"[1.9577294723928658, 1.9325286795067815, 1.907...","[1.7516159531022955, 1.727253336827019, 1.7030...","[1.5534822535715445, 1.5303048180632601, 1.507...","[1.8635025031546752, 1.837012825746756, 1.8105...","[1.6452436545383784, 1.6192891971179828, 1.593...","[1.4325759187629943, 1.4074190188820228, 1.382...","[1.7947810710495278, 1.7672724753875635, 1.739...","[1.567175668568872, 1.5399154757672282, 1.5126...","[1.3422600418310981, 1.3153858280397814, 1.288..."
effarea,"[16.02646574447277, 18.640409974650144, 20.642...","[30.741172868938648, 31.12900829753084, 31.335...","[32.20150639586704, 32.30072738734823, 32.4192...","[23.201936351040672, 25.15330255008084, 27.345...","[31.66688496745329, 31.78564435266949, 31.9733...","[32.79895849992638, 32.88257591265841, 32.9691...","[28.511806683632486, 30.170267208166933, 31.01...","[32.124858046821835, 32.27719909517821, 32.379...","[33.142925415182134, 33.18551353151141, 33.264..."
readnoise,"[6.71848, 6.77015, 6.75663, 6.79981, 7.05629, ...","[6.86434, 6.91468, 6.96803, 7.36224, 7.29712, ...","[8.02339, 8.28742, 8.57657, 8.72061, 8.70782, ...","[7.82572, 8.04383, 7.85935, 7.99911, 8.23975, ...","[6.65295, 6.76853, 6.77448, 6.80252, 7.02615, ...","[8.98901, 9.14457, 9.20017, 9.28653, 9.24306, ...","[8.9218, 9.0226, 9.29537, 9.12591, 9.13382, 9....","[6.57759, 6.50091, 6.55698, 6.63024, 6.69975, ...","[7.25963, 7.36891, 7.41492, 7.48055, 7.45991, ..."


In [171]:
ampList = list(OrderedDict(sorted(raDeg.items())).keys())

In [200]:
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
        
    vendor = det.getSerial()[:3].lower()
    assert dd.vendor[rname].lower() == vendor
    vendorDir = defaultDirs['detector']+'/../'+vendor
    print('Calculating m5 for %s_%s'%(rname,dname))
    
    key = rname+'_'+dname
    for f in filters:
        m5df[key][f] = [-1.]*len(ampList)
    
    for amp in det:
        amp_point = amp.getBBox().getCenter()

        
        try:
            qe_curve = butler.get('qe_curve', raftName=rname, detectorName=dname, taiObs='2000-01-01T00:00:00')
            wavelen = detector0.wavelen 
            sb = qe_curve.evaluate(det, amp_point, wavelen* u.nm, kind='quadratic').value*.01 #unit was percent in CALIB data
            
            if np.max(sb)<0.01:
                print('deal channel: %s, max sb = %.2f'%(key, np.max(sb)))
                continue;
                
            detector.setBandpass(wavelen, sb)
                
            #detector losses  
            #os.listdir(vendorDir)
            detLosses = Bandpass()
            detLosses.readThroughput(os.path.join(vendorDir, '%s_Losses/det_Losses.dat' % (vendor)))
                
            #build hardware and system
            hardware = {}
            system = {}
            for f in filters:
                sb = mirror1.sb * mirror2.sb *mirror3.sb
                sb *= lens1.sb * lens2.sb * lens3.sb * filters[f].sb
                sb *= detector.sb * detLosses.sb
                
                hardware[f] = Bandpass()
                hardware[f].setBandpass(wavelen, sb)
                system[f] = Bandpass()
                system[f].setBandpass(wavelen, sb * atmos.sb)
                
            #calculate m5      
            iamp = ampList.index(amp.getName())
            effarea = adf[key]['effarea'][iamp]*100**2 #convert to cm^2
            readnoise = adf[key]['readnoise'][iamp]
        
            m5 = st.makeM5(hardware, system, darksky=None, 
                      exptime=15, nexp=2, readnoise=readnoise, othernoise=0, darkcurrent=0.2,
                      effarea=effarea, X=1.0)
            for f in filters:
                m5df[key][f][iamp] = m5.m5[f]
        except NoResults:
            print('No results found for this detector')
            continue  # No results found for this detector

Calculating m5 for R10_S00
deal channel: R10_S00, max sb = 0.00
Calculating m5 for R10_S01
Calculating m5 for R10_S02
Calculating m5 for R10_S10
Calculating m5 for R10_S11
Calculating m5 for R10_S12
Calculating m5 for R10_S20
Calculating m5 for R10_S21
Calculating m5 for R10_S22


In [201]:
m5df

,R10_S00,R10_S01,R10_S02,R10_S10,R10_S11,R10_S12,R10_S20,R10_S21,R10_S22
u,"[23.41158847647214, 23.54194712203773, 23.6338...","[23.87849329042126, 23.882729681782106, 23.876...","[23.93719055824004, 23.916488759700542, 23.904...","[23.74627554671593, 23.798243309855458, 23.888...","[24.044128901399212, 24.036119063498752, 24.03...","[23.81417488669588, 23.802394198490845, 23.794...","[23.707740672845013, 23.747976145664396, 23.74...","[23.962227272286466, 23.968479069961703, 23.96...","[24.119577601162202, 24.10827122043142, 24.103..."
y,"[22.106866504239342, 22.196442717735096, 22.25...","[22.49182134744946, 22.502950205425126, 22.506...","[22.48728582164555, 22.490046811091656, 22.499...","[22.282019372537203, 22.329056907489957, 22.38...","[22.484039985938715, 22.487258562291583, 22.49...","[22.524021626246665, 22.52393713801849, 22.524...","[22.420893406350906, 22.45726969470751, 22.472...","[22.512920257511688, 22.519325643967548, 22.52...","[22.508101448160176, 22.510014436116833, 22.51..."
g,"[24.399220940664897, 24.503854077027228, 24.57...","[24.864093728838544, 24.87415814949605, 24.875...","[24.827751838931032, 24.82182665412805, 24.824...","[24.60552976603486, 24.652966263162014, 24.720...","[24.865618819142767, 24.86592742671989, 24.872...","[24.81456759761375, 24.811399172048237, 24.810...","[24.726331199671822, 24.76338538346728, 24.772...","[24.904003167279992, 24.909964103976684, 24.91...","[24.882403905077545, 24.879690200436425, 24.87..."
r,"[23.970203594854127, 24.06512262266343, 24.130...","[24.368338627686356, 24.377616299849798, 24.37...","[24.382842055384714, 24.379358121817887, 24.38...","[24.1859967609663, 24.23167972947961, 24.29022...","[24.396418506901885, 24.396882037271688, 24.40...","[24.374805867919754, 24.37289203696074, 24.371...","[24.279380901948855, 24.313817207985615, 24.32...","[24.4001531984389, 24.403943754618084, 24.4067...","[24.42431641917024, 24.421916177155538, 24.420..."
z,"[22.995485960718064, 23.085684888725915, 23.14...","[23.379172658364812, 23.388560155510277, 23.39...","[23.387784917967885, 23.388975653734207, 23.39...","[23.185618497448765, 23.23221809646433, 23.286...","[23.386455295885348, 23.388533652139472, 23.39...","[23.401019659846952, 23.40188772957844, 23.402...","[23.311356864644175, 23.346311562017178, 23.35...","[23.410785800504705, 23.414456598066636, 23.41...","[23.414373142881097, 23.41454321632211, 23.415..."
i,"[23.552882766121797, 23.64566136721045, 23.709...","[23.938170974875653, 23.94719115888718, 23.948...","[23.958920586031287, 23.957622110281825, 23.96...","[23.756588532341674, 23.803304934068596, 23.85...","[23.963945916170836, 23.965015602660387, 23.96...","[23.958207763992178, 23.957609306741126, 23.95...","[23.867599615650782, 23.902458012810456, 23.91...","[23.970374165129336, 23.973578854157875, 23.97...","[23.98846964884649, 23.987360069487394, 23.987..."


In [192]:
m5df[key][f] 

nan

In [191]:
type(m5df[key][f][0])

TypeError: 'float' object is not subscriptable

In [190]:
type(m5.m5[f])

numpy.float64